##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Состязательный пример использования FGSM

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/adversarial_fgsm"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/adversarial_fgsm.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/adversarial_fgsm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/adversarial_fgsm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве показан пример *состязательной атаки* с использованием Fast Gradient Sign Method (FGSM), как описано в [Объяснение и использование состязательных атак](https://arxiv.org/abs/1412.6572) Гудфеллоу *и др.*. Это была одна из первых и самых популярных атак с целью обмануть нейронную сеть.

## Что такое состязательный пример?

Состязательные атаки - это специализированные входные данные, созданные с целью запутать нейронную сеть, что приводит к неправильной классификации заданного входа. Эти специализированные входные данные неразличимы для человеческого глаза, но сеть не может правильно идентифицировать содержимое изображения. 
Существует несколько типов таких атак, в данном руководстве мы будем рассматривать атаку *белого ящика* методом FGSM, цель которой - гарантировать неправильную классификацию. Атака *белого ящика* - это когда злоумышленник имеет полный доступ к атакуемой модели. Один из самых известных примеров состязательного изображения, показанный ниже, взят из вышеупомянутой статьи.

![Пример состязательности](images/adversarial_example.png)

В этом примере, в исходное изображение *панды* злоумышленник добавляет небольшой шум(искажения), в результате чего модель с высокой степенью вероятности маркирует это изображение как *гиббон*. Ниже объясняется процесс добавления этих шумов.



## FGSM

FGSM работает с использованием градиентов нейронной сети для создания состязательной атаки. Метод применяет градиенты потерь к входному изображению, чтобы создать новое изображение, которое увеличит потери. Этот новый образ называется состязательным. Это можно представить с помощью следующего выражения:
$$adv\_x = x + \epsilon*\text{sign}(\nabla_xJ(\theta, x, y))$$

где

* adv_x: состязательное изображение(результат).
* x: Исходное входное изображение.
* y: Исходная метка входного изображения.
* $\epsilon$: множитель для обеспечения небольшого шума.
* $\theta$: Параметры модели.
* $J$: Потери.

Интересным свойством является тот факт, что градиенты берутся по отношению ко входному изображению. Это сделано для того,чтобы создать изображение, которое увеличит потери. Для этого нужно определить, насколько каждый пиксель изображения влияет на величину потерь, и соответственно добавить шум. Это работает довольно быстро, потому что всего-лишь нужно посчитать, как каждый входной пиксель влияет на потери, используя правило цепочки и находя требуемые градиенты. Поскольку модель больше не обучается, параметры модели остаются постоянными, и таким образом, градиент не расчитывается относительно обучаемых переменных. Единственная цель - обмануть уже обученную модель.

Итак, давайте попробуем обмануть предварительно обученную модель. В этом руководстве используется модель [MobileNetV2](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications/MobileNetV2), предварительно обученная на [ImageNet](http://www.image-net.org/).

In [ ]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

Загрузим предварительно обученную модель MobileNetV2 и имена классов ImageNet.

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet метки
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [ ]:
# Вспомогательная функция для предварительной обработки изображения, чтобы его можно было ввести в MobileNetV2.
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Вспомогательная функция для извлечения меток из вектора вероятности
def get_imagenet_label(probs):
  return decode_predictions(probs, top=1)[0][0]

## Исходное изображение
Давайте воспользуемся образцом изображения [Лабрадора ретривера](https://commons.wikimedia.org/wiki/File:YellowLabradorLooking_new.jpg) Мирко [CC-BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/) из Wikimedia Common и создадим на его основе состязательные примеры. Первым шагом является предварительная обработка, чтобы его можно было использовать в качестве входных данных для модели MobileNetV2.

In [ ]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image_raw = tf.io.read_file(image_path)
image = tf.image.decode_image(image_raw)

image = preprocess(image)
image_probs = pretrained_model.predict(image)

Посмотрим на изображение.

In [ ]:
plt.figure()
plt.imshow(image[0]*0.5+0.5) # To change [-1, 1] to [0,1]
_, image_class, class_confidence = get_imagenet_label(image_probs)
plt.title('{} : {:.2f}% Confidence'.format(image_class, class_confidence*100))
plt.show()

## Создание состязательного образа

### Реализация метода быстрого градиента признаков
Первый шаг - создать шум, который будет использоваться для искажения исходного изображения, что приведет к появлению состязательного изображения. Как уже упоминалось, для этой задачи градиенты берутся по отношению к изображению.

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # Получаем градиенты потерь относительно входного изображения.
  gradient = tape.gradient(loss, input_image)
  # Получаем признаки градиентов, чтобы создать шум
  signed_grad = tf.sign(gradient)
  return signed_grad

Возникающие шумы также можно визуализировать.

In [ ]:
# Получаем метку входного изображения.
labrador_retriever_index = 208
label = tf.one_hot(labrador_retriever_index, image_probs.shape[-1])
label = tf.reshape(label, (1, image_probs.shape[-1]))

perturbations = create_adversarial_pattern(image, label)
plt.imshow(perturbations[0]*0.5+0.5); # Изменяем [-1, 1] на [0,1]

Давайте попробуем разные значения epsilon и посмотрим на полученное изображение. Вы заметите, что по мере увеличения значения epsilon, становится легче обмануть сеть. Однако в результате увеличения epsilon, шумы становятся более заметными.

In [ ]:
def display_images(image, description):
  _, label, confidence = get_imagenet_label(pretrained_model.predict(image))
  plt.figure()
  plt.imshow(image[0]*0.5+0.5)
  plt.title('{} \n {} : {:.2f}% Confidence'.format(description,
                                                   label, confidence*100))
  plt.show()

In [ ]:
epsilons = [0, 0.01, 0.1, 0.15]
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                for eps in epsilons]

for i, eps in enumerate(epsilons):
  adv_x = image + eps*perturbations
  adv_x = tf.clip_by_value(adv_x, -1, 1)
  display_images(adv_x, descriptions[i])

## Следующие шаги

Теперь, когда вы знаете о состязательных атаках, попробуйте их использовать на разных наборах данных и на разных архитектурах. Вы можете создать и обучить свою собственную модель, а затем попытаться обмануть ее, используя тот же метод. Вы также можете посмотреть, как меняется уверенность в предсказаниях при изменении epsilon.

Несмотря на свою мощь, атака, показанная в этом руководстве, была только началом исследования состязательных атак, и с тех пор было опубликовано несколько статей, в которых создавались более мощные атаки. Помимо состязательных атак, исследования также привели к созданию средств защиты, направленных на создание надежных моделей машинного обучения. Вы можете просмотреть этот [обзорный документ](https://arxiv.org/abs/1810.00069) для получения исчерпывающего списка состязательных атак и средств защиты.

Для многих других реализаций состязательных атак и защит вы можете просмотреть библиотеку состязательных примеров [CleverHans](https://github.com/tensorflow/cleverhans).